### Making the necessary imports

In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from statistics import mean, stdev

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import load_model


### Disable scientific notations for clarity

With the parameter "__supress__" set to __True__ we make sure to print floating point numbers using fixed floating
points.

In [ ]:
np.set_printoptions(suppress=True)


### Define useful variables

These variables will be used for saving the ploted data. They must be the same used in the __CNN_Model__ notebook.

In [ ]:
batch_size = 16
epochs = 25
model_version = 3

### Read the CSV file containing the labels of the images

In [ ]:
data = pd.read_csv('dataset.csv')

### Store the important data in variables

The data we are using is:
- "__id__": the name of the correponding image file.
- "__predominant stress__": the number that determines if a leaf is healthy or not and the severity of the disease.
This value ranges from 0 (healthy) to 4 (very diseased).

In [ ]:
data_id = data['id'].tolist()
data_stress = data['predominant_stress'].tolist()

print(data_id)
print(data_stress)

### Create the validation list from the CSV data

We go through all testing images and check their correspondent __id__ in the data we extracted from the CSV.

When an item is found, we check its __predominant stress__ value and if it's 0, it's a healthy leaf and we add to the
validation list "0", but if it's greater than 0 it's a diseased leaf and we add to the validation list "1".

In [ ]:
validation = list()
for file in os.listdir(os.path.join('Dataset/testing', '')):
    split = file.split('.')
    file_id = split[0]

    if int(file_id) in data_id:
        id_index = data_id.index(int(file_id))
        validation.append(0 if data_stress[id_index] == 0 else 1)

print(validation)

### Load the model

This can be a model you created by fiddling with the __CNN_Model__ notebook or can be the provided __best_model.h5__
model.

Either case, make sure to change the path for the __.h5__ file according to your needs.

In [ ]:
model = load_model(f'Models/Batch Size {batch_size} - Epochs {epochs} - Model v{model_version}.h5')


### Make the predictions

Here we set a *__class_names__* variable that stores the name of the classes to identify.

Then we read through all the images in the test folder, loading it and setting its size to be the same that we used to
train our model, then converting this loaded image to an array and expanding its dimensions.

The "*__expand_dims__*" method essentially creates a batch. What it does is adding a new dimension to the array in a
given axis. In this case, it is creating a new dimension on the "**image_array**" in its axis "**0**".

Then we use our loaded model to make a prediction on that image saving it on the **preds** variable.

To compute the score, we use the *__softmax__* function in the first element of the **preds** variable, which contains
the percentage of chance our model calculated for the image being of each class. The softmax is going to normalize the
array and make the sum of its elements be **1**.

Finally we use the *__argmax__* method, which returns the index of the largest element in the **score** list and we
check with our **class_names** variable if this index is 0, corresponding to "**healthy**" or if it's 1, corresponding
to "**diseased**", adding 0 or 1 to the **predictions** list, respectively.

In [ ]:
class_names = ['diseased', 'healthy']
predictions = list()
for file in os.listdir(os.path.join('Dataset/testing', '')):
    img_path = os.path.join('Dataset/testing', file)
    img = keras.preprocessing.image.load_img(img_path, target_size=(512, 512))
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)  # Create a batch

    preds = model.predict(img_array)
    score = tf.nn.softmax(preds[0])
    predictions.append(0 if class_names[np.argmax(score)] == 'healthy' else 1)

print(predictions)

### Calculate metrics scores

For the metrics, we are using four methods of evaluation:
- *__Accuracy__*: $\frac{rights}{total}$, which is the fraction of correct predictions.
- *__Precision__*: $\frac{tp}{(tp + fp)}$, where $tp$ is the number of true positives and $fp$ the number of false
positives.
- *__Recall__*: $\frac{tp}{(tp + fn)}$, where $tp$ is the number of true positives and $fn$ the number of false negatives.
- *__F1__*: $\frac{2 * (precision * recall)}{(precision + recall)}$, which is basically a weighted average of
**precision** and **recall** scores.

After these values are calculated, we calculate the *__mean__* of all metrics and their *__standard deviation__*.

In [ ]:
accuracy = accuracy_score(validation, predictions)
precision = precision_score(validation, predictions, average='binary')
recall = recall_score(validation, predictions, average='binary')
f1 = f1_score(validation, predictions, average='binary')

mean = mean([accuracy, precision, recall, f1])
std_dev = stdev([accuracy, precision, recall, f1])

print(f'Batch Size - {batch_size} | Epochs - {epochs} | Model v{model_version}')
print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1: {f1:.4f}')
print(f'Mean: {mean:.4f}')
print(f'Standard Deviation: {std_dev:.4f}')

### Save metrics to file

Finally, we save all these metrics to a TXT file, which is updated every time new tests are made.

In [ ]:
with open('Accuracy Scores.txt', 'a') as file:
    file.write(f'Batch Size - {batch_size} | Epochs - {epochs} | Model v{model_version}\n')
    file.write(f'Accuracy: {accuracy:.4f}\n')
    file.write(f'Precision: {precision:.4f}\n')
    file.write(f'Recall: {recall:.4f}\n')
    file.write(f'F1: {f1:.4f}\n')
    file.write(f'Mean: {mean:.4f}\n')
    file.write(f'Standard Deviation: {std_dev:.4f}\n\n')
